In [3]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision.datasets import MNIST
import math
from functools import partial

### Get data
Same as previous notebook, condensed with less prints

In [4]:
# Load, reshape, put to float32
trainset = MNIST('../', download=True, train=True)
testset = MNIST('../', download=True, train=False)
train = trainset.data.reshape(60000, -1).to(torch.float32)
test = testset.data.reshape(10000, -1).to(torch.float32)

# Normalize
m, std = train.mean(), train.std()
train = (train - m) / std
test = (test - m) / std

# Get labels
y_train = trainset.targets
y_test = testset.targets

### Define useful classes

In [26]:
class Model(nn.Module):
    def __init__(self, n_in, n_out, depth):
        super().__init__()
        IN = np.linspace(n_in, n_out, depth).astype(int) #quick 'n dirty way to create wider models
        self.layers = []
        for layer in range(depth-2):
            self.layers += [nn.Linear(IN[layer], IN[layer+1])]
            self.layers += [nn.ReLU()]
        self.layers += [nn.Linear(IN[-2],n_out)] #last layer doesn't have relu
        
        
        self.loss = nn.functional.cross_entropy # cross-entropy is nll(log_softmax), as we defined in previous nb
        
    def __call__(self, x):
        for l in self.layers: 
            x = l(x)
        return x

In [27]:
n_in = train.shape[1]
n_layers = 6 # now we can just change the number of layers here 
n_out = 10

learner = Model(n_in, n_out, n_layers)
np.linspace(n_in, n_out, n_layers).astype(int)

array([784, 629, 474, 319, 164,  10])

In [29]:
learner.loss(learner(train), y_train)

tensor(2.3032, grad_fn=<NllLossBackward>)

### Introducing batches
so far we've been training the model with the whole data at every epoch. Most of the time this won't be possible (all datasets aren't nearly as small as MNIST). Thus we split the data in small batches (usually 32, or 64)

In [30]:
bs = 64
EPOCHS = 5
lr = 0.3

for epoch in range(EPOCHS):
    for i in range((60000-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xbatch = train[start_i:end_i]
        ybatch = y_train[start_i:end_i]
        output = learner(xbatch) 
        loss = learner.loss(output, ybatch)

        loss.backward()
        with torch.no_grad():
            for l in learner.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias   -= l.bias.grad   * lr
                    l.weight.grad.zero_()
                    l.bias  .grad.zero_()
    print(loss)

tensor(0.0484, grad_fn=<NllLossBackward>)
tensor(0.0021, grad_fn=<NllLossBackward>)
tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0004, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)


In [9]:
def accuracy(pred, targs):
    return float((pred.argmax(dim=-1) == targs).sum()) / float(len(pred))

In [31]:
output = learner(test)

In [32]:
accuracy(output, y_test)

0.9764

#### So we have a training loop that works
Moreover, last notebook we needed 35 passes through the whole data to get 90% accuracy. With minibatches, the learning is smoother, the model learns more and does so more quickly.

Now we'll refactor the code to make it more maintainable (Note that I could've done that from the start, I just wanted to show the potential recruiter that I know what happens behind the scenes)

# Refactoring the training loop

## updating the parameters

In [112]:
# Define our layers outside the model and pass them as parameters:
depth = 5
IN = np.linspace(n_in, n_out, depth).astype(int) #quick 'n dirty way to create wider models
layers = []
for layer in range(depth-2):
    layers += [nn.Linear(IN[layer], IN[layer+1])]
    layers += [nn.ReLU()]
layers += [nn.Linear(IN[-2],n_out)] #last layer doesn't have relu

In [114]:
IN

array([784, 590, 397, 203,  10])

In [130]:
# Now we can refactor the Model() class and add each layer as a module

class Model(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers
        for i,l in enumerate(layers): self.add_module(f'layer_{i}', l)
            
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [144]:
learner = Model(layers)

In [145]:
# the add_module of the nn.Module class provides us automated access to the model parameters
learner.parameters(), learner.parameters

(<generator object Module.parameters at 0x7f16f1639c00>,
 <bound method Module.parameters of Model(
   (layer_0): Linear(in_features=784, out_features=590, bias=True)
   (layer_1): ReLU()
   (layer_2): Linear(in_features=590, out_features=397, bias=True)
   (layer_3): ReLU()
   (layer_4): Linear(in_features=397, out_features=203, bias=True)
   (layer_5): ReLU()
   (layer_6): Linear(in_features=203, out_features=10, bias=True)
 )>)

In [146]:
# So we can refactor our training loop like so : 

bs = 64 
EPOCHS = 4 # let's speed up the process a little bit, since we aren't using GPU yet
lr = 0.3

for epoch in range(EPOCHS):
    for i in range((60000-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xbatch = train[start_i:end_i]
        ybatch = y_train[start_i:end_i]
        output = learner(xbatch) 
        loss = nn.functional.cross_entropy(output, ybatch)

        loss.backward()
        with torch.no_grad():
            for p in learner.parameters():
                p -= p.grad * lr
                p.grad.zero_()
    print(loss)

tensor(0.0015, grad_fn=<NllLossBackward>)
tensor(9.5367e-05, grad_fn=<NllLossBackward>)
tensor(0.0004, grad_fn=<NllLossBackward>)
tensor(5.8293e-05, grad_fn=<NllLossBackward>)


In [ ]:
# So with self.add_module() in the Model Class, 
# we managed to refactor this part of our training loop:

#for l in learner.layers:
#    if hasattr(l, 'weight'):
#        l.weight -= l.weight.grad * lr
#        l.bias   -= l.bias.grad   * lr
#        l.weight.grad.zero_()
#        l.bias  .grad.zero_()

# into:

#for p in learner.parameters():
#    p -= p.grad * lr
#    p.grad.zero_()

# We had to add code into Model(), but in fact, all it does can be done with nn.Sequential:

In [190]:
# We'll do this to avoid duplicate code every time we refactor the code
def get_linear():  
    learner = nn.Sequential(
    nn.Linear(784, 590),
    nn.ReLU(),
    nn.Linear(590, 397),
    nn.ReLU(),
    nn.Linear(397, 203),
    nn.ReLU(),
    nn.Linear(203, 10)
    )
    return learner

In [189]:
learner = get_linear()

bs = 64 
EPOCHS = 4 # let's speed up the process a little bit, since we aren't using GPU yet
lr = 0.3

for epoch in range(EPOCHS):
    for i in range((60000-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xbatch = train[start_i:end_i]
        ybatch = y_train[start_i:end_i]
        output = learner(xbatch) 
        loss = nn.functional.cross_entropy(output, ybatch)

        loss.backward()
        with torch.no_grad():
            for p in learner.parameters():
                p -= p.grad * lr
                p.grad.zero_()
    print(loss)

tensor(0.0269, grad_fn=<NllLossBackward>)
tensor(0.0081, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0009, grad_fn=<NllLossBackward>)


## iterating through the data

In [191]:
# this part is also a bit messy:
#  
#for i in range((60000-1)//bs + 1):
#    start_i = i*bs
#    end_i = start_i+bs
#    xbatch = train[start_i:end_i]
#    ybatch = y_train[start_i:end_i]

### Dataset and Dataloader classes

In [192]:
class Dataset():
    def __init__(self, x, y): self.x, self.y = x, y
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i], self.y[i]

In [193]:
my_data = Dataset(train, y_train)

In [194]:
# with this, we can the the xbatch and ybatch in one line:
xbatch, ybatch = my_data[0:64]

# But we would still have to calculate the indices each time:
# for i in range((n-1)//bs + 1):
#    start_i = i*bs
#    end_i = start_i+bs

# ideally we'd want a one liner like this one:

# for xb, yb in data:

In [195]:
class DataLoader():
    def __init__(self, data, bs): self.data, self.bs = data, bs
    def __iter__(self):
        for i in range(0, len(self.data), self.bs): yield self.data[i:i+bs]

In [196]:
train_dl = DataLoader(my_data, 64)

In [197]:
for xb, yb in train_dl:
    print(xb.shape, yb.shape)
    break 
#if you remove the break and scroll down you'll see the last bs is 32. 
#we'll take a closer look at this later

torch.Size([64, 784]) torch.Size([64])


In [198]:
# Simplified training loop:
learner = get_linear()
my_data = Dataset(train, y_train)
train_dl = DataLoader(data=my_data, bs=64)

for epoch in range(EPOCHS):
    for xb, yb in train_dl:
        output = learner(xb) 
        loss = nn.functional.cross_entropy(output, yb)

        loss.backward()

        with torch.no_grad():
            for p in learner.parameters():
                p -= p.grad * lr
                p.grad.zero_()
    print(loss)

tensor(0.0072, grad_fn=<NllLossBackward>)
tensor(0.0026, grad_fn=<NllLossBackward>)
tensor(0.0006, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)


## Update the parameters using a class

In [199]:
# Now we want to simplify the training loop further by getting rid of this loop:

# with torch.no_grad():
#     for p in learner.parameters():
#         p -= p.grad * lr
#         p.grad.zero_()



In [200]:
class Optimizer():
    def __init__(self, parameters, lr): self.parameters, self.lr = list(parameters), lr
    
    def step(self):
        with torch.no_grad():
            for p in self.parameters: p -= p.grad * self.lr
    
    def zero_grad(self):
        for p in self.parameters: p.grad.zero_()

In [203]:
EPOCHS = 4
learner = get_linear()
opt = Optimizer(learner.parameters(), 0.4)
my_data = Dataset(train, y_train)
train_dl = DataLoader(data=my_data, bs=64)

for epoch in range(EPOCHS):
    for xb, yb in train_dl:
        output = learner(xb) 
        loss = nn.functional.cross_entropy(output, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    print(loss)

tensor(0.0142, grad_fn=<NllLossBackward>)
tensor(0.0063, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0051, grad_fn=<NllLossBackward>)


That's it! We have a nice and compact training loop. I said we'd add convolutions but this notebook is getting long so we'll do that in the next notebook, along with "in-training" validation to monitor overfitting